## Ingestion del archivo "person.json"

#### `Paso 0 - Actualización de variables y funciones Y validaciones

In [0]:

dbutils.widgets.text("p_environment", "")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
#Chama a função para validar se o arquivo a ser processado existe
valida_arquivo(bronze_folder_path, v_file_date, "person.json")

#### Paso 1 - Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
#%fs
#ls /mnt/moviehistoryadilmor/bronze

In [0]:
#display(spark.read.option("header", True).json("/mnt/moviehistoryadilmor/bronze/person.json"))

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
#Definindo o schema na variavel name_schema
name_schema = StructType(fields=[
    StructField("forename", StringType(), True),
    StructField("surname", StringType(), True)
])

In [0]:
#Definindo o schema na variavel persons_schema
persons_schema = StructType(fields=[
    StructField("personId", IntegerType(), False),
    StructField("personName", name_schema)
])

In [0]:
persons_df = spark.read \
               .schema(persons_schema) \
               .json(f"{bronze_folder_path}/{v_file_date}/person.json")

In [0]:
#display(persons_df)

#### Paso 2 - Renombrar el nombre de las columnas y añadir "ingestion_date" y "enviroment"
1. "personid" renombrar a "person_id"
2. Agregar las columnas "ingestion_date" y "Enviroment"
3. Agragar la columna "name" a partir de la concatenación de "forename" y "surname"
4. Eliminar columnas no requeridas

In [0]:
from pyspark.sql.functions import current_timestamp, lit, col, concat

In [0]:
persons_with_columns_df = add_ingestion_date(persons_df) \
                    .withColumnRenamed("personId", "person_id") \
                    .withColumn("enviroment", lit(v_environment)) \
                    .withColumn("file_date", lit(v_file_date)) \
                    .withColumn("name", 
                                concat(
                                    col("personName.forename"), 
                                    lit(" "), 
                                    col("personName.surname")
                                      )
                                )

In [0]:
persons_final_df = persons_with_columns_df \
                   .select(col("person_id"),
                           col("name"),
                           col("ingestion_date"),
                           col("enviroment"),
                           col("file_date")
                           )

#### Paso 3 - Escribir datos en el datalake en formato parquet

In [0]:
#Elimina partição se ela já existir
#drop_partition_if_exists(persons_final_df, "movie_silver", "persons", "file_date")

In [0]:
#persons_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/persons/")
#persons_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.persons")

#Caso a tabela não existir, cria a tabela com os dados, senão efetua um merge dos dados na tabela
merge_delta_lake(persons_final_df, "movie_silver", "persons", silver_folder_path, "tgt.person_id = src.person_id", "file_date")

In [0]:
#%fs
#ls /mnt/moviehistoryadilmor/silver/persons

In [0]:
#display(spark.read.parquet(f"{silver_folder_path}/persons"))
#spark.sql("select * from movie_silver.persons").display()
display(spark.sql("SELECT file_date, \
                          COUNT(1) \
                   FROM movie_silver.persons \
                   GROUP BY file_date"
                   )
        )

In [0]:
dbutils.notebook.exit("Success")